In [18]:
import os
import logging
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import box, Polygon, MultiPolygon
from shapely.strtree import STRtree

# REF: https://github.com/SaferPlaces2023/safer-buildings → `pip install git+https://github.com/SaferPlaces2023/safer-buildings.git#egg=safer-buildings[with_gdal_py3**]`
from safer_buildings import _consts, _utils
from safer_buildings import filesystem
from safer_buildings import module_retriever, module_s3, module_log

import importlib
_ = importlib.reload(_consts)
_ = importlib.reload(_utils)

In [10]:
module_log.Logger.setLevel(logging.DEBUG)

In [11]:
gdf_wfs = module_retriever.retrieve_venezia_wfs(
    provider = _consts._VENEZIA_WFS_CRITICAL_SITES_PROVIDER,
    bbox = gpd.GeoDataFrame(geometry=[box(-180, -90, 180, 90)], crs="EPSG:4326"),
    buffer_points = True
)

[2025-08-12 18:59:47] [DEBUG   ] ### Retrieved 152797 features from c0107057_grafostrade VENEZIA-WFS service.
[2025-08-12 18:59:47] [DEBUG   ] ### Retrieved 15 features from v_pc_p0104031_prontosoccorso VENEZIA-WFS service.
[2025-08-12 18:59:48] [DEBUG   ] ### Retrieved 768 features from v_pc_p0106011_scuole VENEZIA-WFS service.
[2025-08-12 18:59:48] [DEBUG   ] ### Retrieved 470 features from v_pc_p0106021_stadi VENEZIA-WFS service.
[2025-08-12 18:59:48] [DEBUG   ] ### Retrieved 152 features from v_pc_p0106031_strutturesanitarie VENEZIA-WFS service.
[2025-08-12 18:59:48] [DEBUG   ] ### Retrieved 117 features from v_pc_p0106041_casecura VENEZIA-WFS service.
[2025-08-12 18:59:49] [DEBUG   ] ### Retrieved 164 features from v_pc_p0106051_edificirilevantigenerici VENEZIA-WFS service.
[2025-08-12 18:59:49] [DEBUG   ] ### Retrieved 126 features from v_pc_p0106061_centricommerciali VENEZIA-WFS service.
[2025-08-12 18:59:49] [DEBUG   ] ### Retrieved 179 features from v_pc_p0106071_museibibliote

In [15]:
lotto1_bbox = gpd.read_file('https://s3.us-east-1.amazonaws.com/saferplaces.co/Venezia/shapes/bacino1.shp').to_crs(epsg=4326).total_bounds
lotto2_bbox = gpd.read_file('https://s3.us-east-1.amazonaws.com/saferplaces.co/Venezia/shapes/bacino2.shp').to_crs(epsg=4326).total_bounds
lotto3_bbox = gpd.read_file('https://s3.us-east-1.amazonaws.com/saferplaces.co/Venezia/shapes/bacino3.shp').to_crs(epsg=4326).total_bounds
lotto4_bbox = gpd.read_file('https://s3.us-east-1.amazonaws.com/saferplaces.co/Venezia/shapes/bacino4.shp').to_crs(epsg=4326).total_bounds
lotto5_bbox = gpd.read_file('https://s3.us-east-1.amazonaws.com/saferplaces.co/Venezia/shapes/bacino5.shp').to_crs(epsg=4326).total_bounds

In [16]:
gdf_wfs_1 = gdf_wfs.cx[lotto1_bbox[0]:lotto1_bbox[2], lotto1_bbox[1]:lotto1_bbox[3]]
gdf_wfs_2 = gdf_wfs.cx[lotto2_bbox[0]:lotto2_bbox[2], lotto2_bbox[1]:lotto2_bbox[3]]
gdf_wfs_3 = gdf_wfs.cx[lotto3_bbox[0]:lotto3_bbox[2], lotto3_bbox[1]:lotto3_bbox[3]]
gdf_wfs_4 = gdf_wfs.cx[lotto4_bbox[0]:lotto4_bbox[2], lotto4_bbox[1]:lotto4_bbox[3]]
gdf_wfs_5 = gdf_wfs.cx[lotto5_bbox[0]:lotto5_bbox[2], lotto5_bbox[1]:lotto5_bbox[3]]

print(f'Len gdf_wfs_1: {len(gdf_wfs_1)}')
print(f'Len gdf_wfs_2: {len(gdf_wfs_2)}')
print(f'Len gdf_wfs_3: {len(gdf_wfs_3)}')
print(f'Len gdf_wfs_4: {len(gdf_wfs_4)}')
print(f'Len gdf_wfs_5: {len(gdf_wfs_5)}')

Len gdf_wfs_1: 11726
Len gdf_wfs_2: 74465
Len gdf_wfs_3: 59189
Len gdf_wfs_4: 38539
Len gdf_wfs_5: 40578


In [17]:
gdf_wfs_1.to_file('building_datasets/venezia_wfs_critical_sites_1.gpkg', driver='GPKG')
gdf_wfs_2.to_file('building_datasets/venezia_wfs_critical_sites_2.gpkg', driver='GPKG')
gdf_wfs_3.to_file('building_datasets/venezia_wfs_critical_sites_3.gpkg', driver='GPKG')
gdf_wfs_4.to_file('building_datasets/venezia_wfs_critical_sites_4.gpkg', driver='GPKG')
gdf_wfs_5.to_file('building_datasets/venezia_wfs_critical_sites_5.gpkg', driver='GPKG')

In [19]:
module_s3.s3_upload(
    filename = 'building_datasets/venezia_wfs_critical_sites_1.gpkg',
    uri = 's3://saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_1.gpkg'
)
module_s3.s3_upload(
    filename = 'building_datasets/venezia_wfs_critical_sites_2.gpkg',
    uri = 's3://saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_2.gpkg'
)
module_s3.s3_upload(
    filename = 'building_datasets/venezia_wfs_critical_sites_3.gpkg',
    uri = 's3://saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_3.gpkg'
)
module_s3.s3_upload(
    filename = 'building_datasets/venezia_wfs_critical_sites_4.gpkg',
    uri = 's3://saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_4.gpkg'
)
module_s3.s3_upload(
    filename = 'building_datasets/venezia_wfs_critical_sites_5.gpkg',
    uri = 's3://saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_5.gpkg'
)

[2025-08-12 19:18:09] [DEBUG   ] uploading building_datasets/venezia_wfs_critical_sites_1.gpkg into saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_1.gpkg...
[2025-08-12 19:18:12] [DEBUG   ] uploading building_datasets/venezia_wfs_critical_sites_2.gpkg into saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_2.gpkg...
[2025-08-12 19:18:16] [DEBUG   ] uploading building_datasets/venezia_wfs_critical_sites_3.gpkg into saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_3.gpkg...
[2025-08-12 19:18:19] [DEBUG   ] uploading building_datasets/venezia_wfs_critical_sites_4.gpkg into saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_4.gpkg...
[2025-08-12 19:18:22] [DEBUG   ] uploading building_datasets/venezia_wfs_critical_sites_5.gpkg into saferplaces.co/Venezia/shapes/buildings/venezia_wfs_critical_sites_5.gpkg...


'building_datasets/venezia_wfs_critical_sites_5.gpkg'